In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("dog_data_09032022.csv")
df.tail(3)

## Data Cleaning
- Drop unneccsary columns
- Clean height
    - drop all Nan Rows
    - Choose one unit, male and standard
    

In [ ]:
# Let's tackle one column after another
# drop first unnamed column
df = df.drop(df.columns[0], axis=1)
df.tail(2)
print(df.shape)

In [ ]:
df['temp'] = df.height.apply(lambda x: len(str(x).split('~')))

In [ ]:
# remove all Nan rows
# print(df.shape)
# df = df[df['height'].notna()]
# print(df.shape)

## Height Cleaning

In [ ]:
# we want to get regular, male, standard
def height_separate(height_str):
    height_str = height_str.lower()
    if 'male' in height_str:
        height_str = height_str.split('(male')[0]
    elif 'standard' in height_str:
        height_str_list = height_str.split('~')
        for height_str_iter in height_str_list:
            if '(standard)' in height_str_iter:
                height_str = height_str_iter
        height_str = height_str.split('(standard)')[0]
        
    # remove inches
    height_str = height_str.replace('inches',"")
    height_str = height_str.replace('pounds',"")
    height_str = height_str.replace('~',"")
        
    return height_str

def height_str_to_val(height_str):
    height_str = height_str.split('-')
    height_val_list = []
    for height_digit in height_str:
        try: 
            height_val = float(height_digit)
            height_val_list.append(height_val)
        except:
            return height_str
    return sum(height_val_list) / len(height_val_list)



In [ ]:
df['height'] = df.height.apply(lambda x : height_separate(x) if type(x) == str else "NA")

In [ ]:
df['height'] = df.height.apply(lambda x : height_str_to_val(x))

In [ ]:
h_list = []
for h in df['height']:
    if type(h) == float:
        h_list.append(h)

h_mean = sum(h_list) / len(h_list)
print(h_mean)

In [ ]:
df['height'] = df['height'].apply(lambda x : h_mean if x == ["NA"] else x)

In [ ]:
idx_list = []
for i,h in zip(df.index,df.height):
    if type(h) is not float:
        idx_list.append(i)
        print(h)
print(idx_list)

In [ ]:
df.height.loc[22] = (20.5 + 22) / 2
df.height.loc[26] = 15
df.height.loc[28] = 20
df.height.loc[50] = 28
df.height.loc[73] = (22.5 + 27.5) / 2
df.height.loc[88] = (23.5 + 25.5) / 2
df.height.loc[141] = 32
df.height.loc[154] = 27.5
df.height.loc[168] = 30
df.height.loc[177] = 28
df.height.loc[185] = 22
df.height.loc[192] = (15.75+19.75) / 2
df.height.loc[200] = 15
df.height.loc[201] = 10
df.height.loc[203] = (16 +22) / 2
df.height.loc[212] = 30.5
df.height.loc[232] = (19.5+24.5) / 2
df.height.loc[245] = 15.5
df.height.loc[247] = (28+35) / 2
df.height.loc[259] = 26
df.height.loc[263] = 21.5
df.height.loc[276] = 15.5

## Weight Cleaning

In [ ]:
df['weight'] = df.weight.apply(lambda x : height_separate(x) if type(x) == str else "NA")
df['weight'] = df.weight.apply(lambda x : height_str_to_val(x))

In [ ]:
import statistics
w_list = []
for w in df['weight']:
    if type(w) == float:
        w_list.append(w)

w_mean = sum(w_list) / len(w_list)
print(w_mean)

In [ ]:
df['weight'] = df['weight'].apply(lambda x : w_mean if x == ["NA"] else x)

In [ ]:
idx_list = []
for i,h in zip(df.index,df.weight):
    if type(h) is not float:
        idx_list.append(i)
        print(h)
print(idx_list)

In [ ]:
df.weight.loc[73] = (88 + 110)/2
df.weight.loc[76] = 6
df.weight.loc[88] = (44 + 57)/2
df.weight.loc[96] = 110
df.weight.loc[112] = 28
df.weight.loc[126] = 100
df.weight.loc[154] = 100
df.weight.loc[165] = 7
df.weight.loc[167] = 12
df.weight.loc[189] = 14
df.weight.loc[190] = 30
df.weight.loc[192] = (17.5+26.5) / 2
df.weight.loc[203] = (35+44) / 2
df.weight.loc[222] = 6.5
df.weight.loc[238] = 10
df.weight.loc[265] = 55

## clean life

In [ ]:
def life_cleaning(life_str):
    life_str = str(life_str)
    life_str = life_str.replace('years','')
    life_list = life_str.split('-')
    try:
        life_list = [int(l) for l in life_list]
    except:
        life_list = [0]
    life_avg = sum(life_list) / len(life_list)
    return life_avg

In [ ]:
df.life.value_counts()

In [ ]:
df.life = df.life.apply(lambda x: life_cleaning(x))

In [ ]:
df.life.value_counts()

In [ ]:
# fill in with median life
# 0 : 5
# 1214 : 1

life_avg = (df.life.sum() - 1214) / (len(df.life) - 5)
df['life'] = df['life'].apply(lambda x: life_avg if (x == 0 or x == 1214) else x)


## Trait columns cleaning

In [ ]:
fig = plt.figure(figsize= (15,20))
ax = fig.gca()
df.hist(ax = ax)

In [ ]:
trait_cols = ['Affectionate With Family',
       'Good With Young Children', 'Good With Other Dogs', 'Shedding Level',
       'Coat Grooming Frequency', 'Drooling Level', 'Coat Type', 'Coat Length',
       'Openness To Strangers', 'Playfulness Level',
       'Watchdog/Protective Nature', 'Adaptability Level',
       'Trainability Level', 'Energy Level', 'Barking Level',
       'Mental Stimulation Needs']

def clean_trait(series):
    idxmax = series.value_counts().idxmax()
    series[series == 0] = idxmax # replace 0 with idxmax
    return series
    
for col in trait_cols:
    df[col] = clean_trait(df[col])

In [ ]:
fig = plt.figure(figsize= (15,20))
ax = fig.gca()
df.hist(ax = ax)

## clean popularity rank

In [ ]:
df.popularity_rank = df.popularity_rank.apply(lambda x: int(x.replace(' of 1','')) if type(x) == str else x)
df.popularity_rank.sort_values(ascending= False)
# out of 197
# assign all the Nan to be 198th
df.popularity_rank = df.popularity_rank.where(df.popularity_rank >= 1, 198)

## clean coat_length

In [ ]:
def coat_length_convert(coat_length):

    coat_length = str(coat_length).lower()

    if 'long' in coat_length:
        return 3
    elif 'medium' in coat_length:
        return 2
    elif 'short' in coat_length:
        return 1
    else:
        return 2

df["Coat Length"] = df["Coat Length"].apply(lambda x: coat_length_convert(x))

## clean coat_length

In [ ]:
df['Coat Type'].value_counts()

# smooth, double, others

def coat_type_convert(coat_type):

    coat_type = str(coat_type).lower()

    if 'smooth' in coat_type:
        return 'smooth'
    elif 'double' in coat_type:
        return 'double'
    else:
        return 'others'

df["Coat Type"] = df["Coat Type"].apply(lambda x: coat_type_convert(x))
df["Coat Type"].value_counts()

## color cleaning

In [ ]:
def color_split(color_str):
    color_str = str(color_str)
    color_list = list(set(color_str.replace('-','/').replace(' & ','/').split('/')))
    return color_list

In [ ]:
df.color = df.color.apply(lambda x: color_split(x))

In [ ]:
color_set = {}
for c_list in df.color:
    for c in c_list:
        if c in color_set:
            color_set[c] = color_set[c] +1
        else:
            color_set[c] = 1
pd.Series(color_set).sort_values(ascending = False).head(5)

# Black             180
# White             157
# Red               121
# Tan               100
# Fawn               72

# Above 5 colors are the most popular color in dogs
# let's create black, white, red columns


In [ ]:
df['color'] = df['color'].apply(lambda x : [i.lower() for i in x] )

In [ ]:
def contains_word(color_list,str):
    for c in color_list:
        if str in c:
            return 1
    return 0


In [ ]:
df['black_color'] = df['color'].apply(lambda x: contains_word(x,'black'))
df['white_color'] = df['color'].apply(lambda x: contains_word(x,'white'))
df['red_color'] = df['color'].apply(lambda x: contains_word(x,'red'))

In [ ]:
s = set()
l = []

for c in df.color:
    if type(c) is not list:
        l = l + []
    else:
        l = l + c

for c in df.color:
    if type(c) is list:
        s = s.union(set(c))

## mark cleaning

In [ ]:
df.marking = df.marking.apply(lambda x: x.split('-') if type(x) == str else x) 

In [ ]:
s = set()
l = []

for c in df.marking:
    if type(c) is not list:
        l = l + []
    else:
        l = l + c

for c in df.marking:
    if type(c) is list:
        s = s.union(set(c))


print(len(l))
print(len(s))

In [ ]:
df['marking'] = df['marking'].apply(lambda x: 0 if x is np.nan else 1)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#df = df.drop(['color','health','grooming','excercise','training','nutrition','temp'],axis = 1)
df = pd.get_dummies(df, columns = ['Coat Type'])

In [ ]:
df.to_csv('dog_data_clean.csv')